# The Farmer 101
Author: John Weaver (2019)

Updated in Feb 2021

## This notebook will demonstrate how to use The Farmer to model galaxies 

We assume that you have installed the following packages. It is strongly recommended that you use Python 3 inside a virtual envionment.

Within Python:
- Tractor (http://thetractor.org/, and dependencies within)
- Numpy
- Scipy
- Astropy
- Matplotlib
- Sep (https://sep.readthedocs.io/en/v1.0.x/index.html)
- Pathos (for multi-processing; https://pathos.readthedocs.io/en/latest/index.html)

Standalone:
- SExtractor (https://www.astromatic.net/software/sextractor)
- PSFEx (https://www.astromatic.net/software/psfex)

## Step 1. Configuration

First you must have the following data files:

- An image you wish to detect on
- An image (or images) you wish to model on
- An image (or images) you wish to force those models onto

The detection image and model image may be the same, and you may not have other bands to force photometry onto.

For each of these, you will also want a robust inverse variance weight image, and probably a boolean (1/0) mask file. 1's are valid pixels, 0's are excluded.

Your detection image should be optimized for detection, even if it's not in sensible units (e.g. a stack, color-image, or CHIMEAN). It's just to get the number of sources, a guess location, and a segmentation map.

**Some terminology should be introduced:**
- A **brick** is a cut-out of the larger image, for computational ease.
- A **blob** is a cut-out of a group of nearby sources, again for computational ease.
- **detection** means having to do with the detection image
- **modeling** means having to do with the model image
- **multiband** means having to do with the (likely) multiple forced photometry images

Nearby models must be modelled simultaneously. This is decided by dilating the segmentation map by a morphological structure function, joined, and re-labeled. Sources within this 'blob' are jointly modeled. The models are optimized using a decision tree to determine the model type. The solution models are then optimized together (to ensure a fair optimization) one last time where the model parameters are finalized.

**The model decision tree:**
 
1. PointSource -- or -- SimpleGalaxy
2. ExpGalaxy -- or -- DevGalaxy
3. Composite Galaxy

Models have different profiles and number of parameters:

- PointSource :: basic PSF model. 3 parameters: (x, y) and flux.
- SimpleGalaxy :: For nearly resolved sources. Exponential wings and dev core. 3 parameters: (x, y) and flux.
- ExpGalaxy ::  Exponentially declining profile. 5 parameters: (x,y), flux, position angle, axis-ratio, and effective radius.
- DevGalaxy :: de Vaucouleurs profile. 5 parameters: (x,y), flux, position angle, axis-ratio, and effective radius.
- Composite :: Exponential wings and dev core. 10 parameters: (x, y),  total flux, fraction of flux in DevGalaxy, and position angle, axis-ratio, and effective radius for each Exp and Dev components.

NB: (x,y) may be frozen to the position solution at the PointSource stage in the config file under FREEZE_POSITION.

Optimzation, in brief, proceeds by maximizing likelihood. The likelihood is calculated by summing in quadriture the per-pixel chi values (that is, {data - model} * weight). If your weights are unreliable, so will your models, fluxes, and uncertainties! 

The Farmer provides a great deal of flexibility. However, that means that the user must carefully configure the data and input parameters...

### 1A : The config file

Farmer contains a lengthy configuration file. Most of the parameters can be left alone, but the user should have an idea what they do. The exact parameters are liable to change! 

The default configuration file "config_default.py" contains this information. It is a python script and can utilize pythonic lists, dictionaries, indexing, etc.

**IMPORTANT**: The configuration file which Farmer uses must have the name "config.py" in order to be recognized. More on that in 1B.

#### General Functionality
These are just general configurations which are often changed.

- CONSOLE_LOGGING_LEVEL :: VERBOSE console level ('DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL')
- LOGFILE_LOGGING_LEVEL :: VERBOSE logfile level (same options, but can also be None)
- PLOT                  :: Plotting level (0 is no plots, 3+ is full plotting. Not available when using multiprocessing).
- NTHREADS              :: Number of processes to run simultaneously
- OVERWRITE :: Overwrite existing files without warning?
- OUTPUT :: Write catalog and other files to memory?

#### File Location
Location of various directories containing data or to where data will be written.

- IMAGE_DIR :: Holds the raw images
- PSF_DIR :: Holds the PSF images
- BRICK_DIR :: Intermediate data directory for brick files
- INTERIM_DIR :: Intermediate data directory for auxillary data
- PLOT_DIR :: Intermediate data directory for figures
- CATALOG_DIR :: Output data directory for catalogs
- LOGGING_DIR :: Intermediate directory to contain logging files
- SFDMAP_DIR :: Not used in this version

- STARCATALOG*** :: Not used in this version

#### File Name Patterns, Bands, and Zeropoints
Recongnition patterns for file names and their zeropoints.

- IMAGE_EXT :: Extension pattern for images
- WEIGHT_EXT :: Extension pattern for weights (optional)
- MASK_EXT :: Extension pattern for masks (optional)
- DETECTION_NICKNAME :: Used for output labels
- DETECTION_FILENAME :: Filename pattern for detection image. Fill with 'EXT' to take IMAGE/WEIGHT/MASK_EXT.
- DETECTION_ZPT :: AB Magnitude zeropoint of detection image
- MODELING_NICKNAME :: Used for output labels
- MODELING_FILENAME :: Filename pattern for detection image. Fill with 'EXT' to take IMAGE/WEIGHT/MASK_EXT.
- MULTIBAND_NICKNAME :: Used for output labels
- MULTIBAND_FILENAME :: Filename pattern for multiband images. Fill with 'EXT' to take IMAGE/WEIGHT/MASK_EXT and 'BAND' to take band image name.
- BANDS :: List of band name to fill 'BAND' in MULTIBAND_FILENAME.
- MULTIBAND_ZPT :: List of corresponding zeropoints

#### Image Properties
Basic size and pixel scale of the images. Assumes all images are equal size and pixel-aligned.

- PIXEL_SCALE :: in arcsec/px
- MOSAIC_WIDTH :: Pixel width of images
- MOSAIC_HEIGHT :: Pixel height of images

#### Point-Spread Functions
Parameters to generate and realize the PSF per image.

- CONSTANT_PSF :: List of band names for which a constant PSF should be generated, and realized in the models. Do not change this once you've generated a set of PSFs!
- PRFMAP_PSF :: List of bands for which a PRF should be generated (for IRAC)
- PRFMAP_GRID_FILENAME :: Filename for PRF grid
- PRFMAP_DIR :: Parent directory for PRF materials
- PRFMAP_COLUMNS :: translation of ID, RA, and Dec columns
- PRFMAP_FILENAME :: Filename pattern for PRF files
- USE_BLOB_IDGRID :: If True, assigns a PRF for each blob. Requires custom PRF map grid procedure.
- PRFMAP_MAXSEP = Maximum acceptable separation for PRF match (in pixels)
- PRFMAP_PIXEL_SCALE_ORIG : Original pixel scale of PRFs (for automatic re-sampling)
- PRFMAP_FORCE_SIZE :: Override the size of the PRF
- PRFMAP_MASKRAD :: PRF masking radius
- PSFVAR_NSNAP :: Number of PSF snapshots per axis with PSFEx 
- PSFGRID :: Bands to use for PSF grid technique. Requires custom PSF grid procedure.
- PSFGRID_OUT_DIR :: Parent directory for PSF grid technique materials.
- PSFGRID_MAXSEP :: Maxmimum acceptable separation for PSF match (in pixels)
- USE_STARCATALOG :: If true, uses star catalog to find PSF candidates
- MOD_REF_LIMITS :: Tuple of flux radius (min, max) to select model PSF
- MOD_VAL_LIMITS :: Tuple of apparent magnitude (min, max) to select model PSF
- MULTIBAND_REFF_LIMITS :: List of tuples of flux radius (min, max) to select multiband PSFs. Must have same index as in BANDS.
- MULTIBAND_VAL_LIMITS :: List of tuples of apparent magnitude (min, max) to select multiband PSFs. Must have same index as in BANDS.
- NORMALIZE_PSF :: Force normalization of your PSF? Recommended to be set to 'True'.
- NORMALIZE_THRESH :: Threshold to which tractor is allowed to get the normalization wrong during the FFT convolution.
- RMBACK_PSF :: List of bands (in BANDS) for which the PSF background should be median estimated and have that value subtracted from the PSF.
- PSF_MASKRAD :: Radius in arcsec beyond which PSF background is calculated.
- FORCE_GAUSSIAN_PSF :: Replace PSFs with gaussians generated from the PSF_SIGMA parameter, always. Recommended to be set to 'False'.
- USE_GAUSSIAN_PSF :: Replace PSFs with gaussians generated from the PSF_SIGMA parameter, if the psf is None. Recommended to be set to 'False'.
- PSF_SIGMA :: Standard deviation of Gaussian PSF.


#### Tractor Engine
Parameters that run the Tractor
- DAMPING :: Regularization parameter for least-squares iteration
- TRACTOR_MAXSTEPS :: Maximum number of interations for optimization attempt.
- TRACTOR_CONTHRESH :: Liklihood convergence threshold for optimization.
- USE_RMS_WEIGHTS :: ???
- SCALE_WEIGHTS :: ???
- USE_MASKED_SEP_RMS :: ???
- APPLY_SEGMASK :: Mask flux from soruces outwith the blob during modeling. Recommended to be set to 'True'.
- ITERATIVE_SUBTRACTION_THRESH :: Blobs above this limit will not be fit simultaneously. Dangerous, but sometimes necessary for large blobs.
- CORRAL_SOURCES :: Catch escaping sources and "corral" them back into their segments.
- REFF_MIN :: Minimum allowable effective radius, in arcsec.

#### Debug Mode
Parameters to debug failures
- NBLOBS :: Number of blobs to run.
- TRY_OPTIMIZATION :: If True, a failure from Tractor will not cause a catastrophic crash. Useful for debugging those failures.

#### Modeling
Parameters to drive modeling

- MODELING_BANDS :: List of bands used in modeling
- MODEL_PHOT_MAX_NBLOB :: Maximum number of sources in a blob. Blobs with more will not be run.
- FORCE_POSITION :: True/False. Force the position of the model to always be the detected position. Recommended to be set to 'False'.
- FREEZE_POSITION :: True/False. Freeze the position of the source with the optimal position found with the pointsource model. Speeds up computation.
- FREEZE_FINAL_POSITION :: True/False. Freeze position of sources during final optimization, borrowing from best-fit model. 
- USE_MODEL_POSITION_PRIOR :: If True, the position prior will be applied.
- MODEL_POSITION_PRIOR_SIG :: Sigma of the Gaussian prior on the position, in arcsec.
- USE_MODEL_SHAPE_PRIOR :: If True, the position prior will be applied.
- MODEL_REFF_PRIOR_SIG :: Sigma of the Gaussian prior on the effective radisu, in arcsec.
- USE_BIC :: Use the Bayesian Inference Criterion for decision tree, if not then use Chisq/N.
- USE_SEP_INITIAL_FLUX :: If True, use the SEP fluxes as a start. Only useful if you detection images was in the same units as your other images.

Decision Tree Options
- DECISION_TREE :: 1 (with Exp/Dev) or 2 (with Sersic -- EXPERIMENTAL)

Option 1
- PSF_SG_THRESH1 :: Threshold in Chisq/N or BIC which SG must beat to move to the next tier.
- EXP_DEV_THRESH :: Threshold in Chisq/N or BIC within which it moves to the next tier.
- CHISQ_FORCE_EXP_DEV :: Threshold in Chisq/N below which the source model must beat to stop advancing to the middle tier.
- CHSIQ_FORCE_COMP1 :: Threshold in Chisq/N below which the source must beat to stop advancing to the bottom tier.

Option 2 (Not for use!)
- PSF_SG_THRESH2 :: Threshold in Chisq/N or BIC which SG must beat to move to the next tier.
- CHISQ_FORCE_SERSIC :: Threshold in Chisq/N below which the source model must beat to stop advancing to the middle tier.
- CHISQ_FORCE_SERSICCORE :: Threshold in Chisq/N below which the source model must beat to stop advancing to the bottom tier.

#### Forced photometry
Parameters to control the photometric extraction with models.
- INIT_FLUX_BAND :: Band used as initial guess flux.
- FORCED_PHOT_MAX_NBLOB :: Skip forced photometry on blobs with more sources than this value. For testing.
- FREEZE_FORCED_POSITION :: If False, each band will run by itself and the position will be allowed to vary.
- USE_FORCE_POSITION_PRIOR :: If True, will use a position prior to constrain the forced model.
- FORCE_POSITION_PRIOR_SIG :: Sigma of the Gaussian prior on position, in arcsec.
- FREEZE_FORCED_SHAPE :: If False, each band will run by itself and the shape will be allowed to vary. DANGEROUS.
- USE_FORCE_SHAPE_PRIOR :: If True, will use a shape prior to constrain the forced model.
- FORCE_REFF_PRIOR_SIG :: Sigma of the Gaussian prior on effective radius, in arcsec.
- FORCE_EE_PRIOR_SIG :: Sigma of the Gaussian prior on the sigmoid-convolved ellipticity parameters. Not used currently.


#### Bricks and Blobs
Basic dimensions and blob determination.

 - BRICK_WIDTH :: Size of brick. MOSAIC_WIDTH/BRICK_WIDTH should be an integer.
 - BRICK_HEIGHT :: Size of brick. MOSAIC_HEIGHT/BRICK_HEIGHT should be an integer.
 - BRICK_BUFFER :: Buffer around the maximum dimensions of the blob to determine the cutout size. May be especially useful if APPLY_SEGMAP is False.
 - DILATION_RADIUS :: Radius of morphological structure function used in the dilation.
 - SEGMAP_MINAREA :: Used to recover sources from binary erosion. Not used currently.

#### Source Detection
Basically from SExtractor. Used in sep.

- USE_DETECTION_WEIGHT :: True/False. Use the weight map corresponding to the detection image?
- DETECTION_SUBTRACT_BACKGROUND :: True/False. Subtract the background of the detection image?
- SUBTRACT_BACKGROUND :: True/False. Subtract the background of the model and multiband images?
- SUBTRACT_BACKGROUND_WITH_MASK :: Use segmap to mask sources when determining background. Small correction.
- SUBTRACT_BACKGROUND_WITH_DIRECT_MEDIAN :: Not used in this version.
- MANUAL_BACKGROUND :: Manual background values for each band. Dictionary.
- SAVE_BACKGROUND :: Write background to FITS file.
- DETECT_BW :: Background mesh width for the detection image.
- DETECT_BH :: Background mesh height for the detection image.
- DETECT_FW :: Background filter width for the detection image.
- DETECT_FH :: Background fitler height for the detection image.
- SUBTRACT_BH :: Background mesh width for the model/multiband image.
- SUBTRACT_BW :: Background mesh height for the model/multiband image.
- SUBTRACT_FW :: Background filter width for the model/multiband image.
- SUBTRACT_FH :: Background filter height for the model/multiband image.
- USE_FLAT :: True/False. Use the flat (global) background level, or else use the varying background estimate.
- THRESH :: Detection threshold (absolute if USE_DETECTION_WEIGHT is False)
- MINAREA :: Minimum contiguous area above threshold to centoid.
- FILTER_KERNEL :: SExtractor filter kernel filename, must be in config/ directory.
- FILTER_TYPE :: Filter kernel type.
- DEBLEND_NTHRESH :: Deblending threshold
- DEBLEND_CONT :: Deblending continuous threshold.
- PIXSTACK_SIZE :: Size of pixel stack allowed in memory.

#### Aperture Photometry
Measure apertures on your image, as well as models and residuals - on the fly!

- DO_APPPHOT :: Perform simple aperture photometry on image, model, isolated model, and residual?
- APER_APPLY_SEGMASK :: Apply segmentation mask when doing aperture photometry?
- APER_PHOT :: Diameter (in arcsec) of apertures
- DO_SEPHOT :: Perform advanced aperture photometry on image, model, isolated model, and residual with SEP? (I.e. MAG_AUTO)
- PHOT_AUTOPARAMS :: Kron factor, in units of pixels 
- PHOT_FLUXFRAC :: Fraction of FLUX_AUTO used for defining FLUX_RADIUS

#### Residual Source Detection
Find out if anything is leftover after the model is subtracted. Experimental.

- DO_SEXPHOT :: True/False. Perform on-the-fly detection on residuals?
- RES_THRESH :: Detection threshold on residual
- RES_MINAREA :: Detection mininum area on residual
- RES_DEBLEND_NTHRESH :: Deblending threshold on residual
- RES_DEBLEND_CONT :: Deblending continuous threshold

#### Miscellaneous
Can ignore most of this stuff.

- X_COLNAME :: Column name of x position from external catalog to force centoids (optional).
- Y_COLNAME :: Column name of y position from external catalog to force centroids (optional).
- MAKE_MODEL_IMAGE :: True/False. Generate the brick as a model?
- MAKE_RESIDUAL_IMAGE :: True/False. Generate the brick as a residual?
- RESIDUAL_CHISQ_REJECTION :: Chisq/N threshold below which models are included in the map generation.
- RESIDUAL_NEGFLUX_REJECTION :: True/False. Exclude models which have negative flux from map generation?
- RESIDUAL_AB_REJECTION :: True/False. Exclude models which have a crazy axis ratio?
- SPARSE_SIZE :: Area of blob in pixels which may be candidates for artifacts (i.e. cosmic ray -- it will likely be a giant blob cutout with an elongated geometry, with mostly non-blob pixels.)
- SPARSE_THRESH :: Blob is skipped if ratio of masked area to the entire blob cutout exceeds this threshold.

### 1B : Where data lives

The ideal data structure for The Farmer is as follows:

Your_Farmer_Directory/

    -> data/

        -> external/ (downloads, not to be altered)

        -> images/ (your mosaics, weights, and masks)

        -> intermediate/ (middle stuff)

            -> bricks/ (detection, modelling, and multiband bricks)

            -> interim/ (segmentation and auxillary maps)

            -> plots/ (your plots)

            -> psfmodels/ (.psf models)

        -> output

            -> catalogs/ (the prize!)

    -> config/

        -> config.py

        -> translate.py

        -> config.psfex

        -> config_psfex.sex

        -> param_psfex.sex

        -> conv_filters/

You can use a convenience function to make the directories for you: (see the import invocation in 2A)



In [ ]:
interface.make_directories()

## Step 2. Data Pre-processing

Once the data has been assembled into data/images/, it must be renamed. This naming convention makes it easier for you to reproduce your work without relying on several bash scripts that you keep around. In esssence, you are asked to supply a filename pattern, which features an 'EXT' key and for the multiband, a 'BAND' key. These are then replaced by the EXT keywords for the image, weight, and mask when The Farmer goes looking for your data. If it doesn't find a weight or mask, it assumes equal weights or no masking, respectively.

The Farmer also must be configured through the **translate.py** file. This file allows you to connect an easy to remember (and label) image band name with its true file name. 

For example, if the full filename is 'sdss_z_allsky_gaia_image.fits', then one might write EXT_IMAGE='_image.fits' and BANDS=['sdss_z',]. Then in the translate file have 'sdss_z':'sdss_z_allsky_gaia'.

Once your configuration file is setup and data in place, you are ready to start Farming!

### 2A: The interface

The Farmer is accessible through a minimal python interface, which is still a work in progress. To begin, you must be in the directory above config/ so The Farmer knows where your configuration file is. Then:

In [ ]:
from src.core import interface

Calling the interface will trigger The Farmer to read through your configuration file
and check the translation file for existing files. It will warn you if it cannot find a cerain file that it expects. So long as you have configured the verbosity to report to you about INFO or DEBUG, it will let you know if it is successful at this stage.

### 2B: The Bricks
A Brick is a section of the larger mosaic which contains some sources. There are three kinds of bricks, corresponding to your detection, modelling, and multiband images. A brick includes your science image, weight, and mask. It will always include a weight and mask, even if you do not supply one. In that case, the weights will be all 1's and the mask will be all 0s. 

The brick is designed such that it has a BRICK_BUFFER. The role of the buffer is to allow centroids to be placed inside the image, but not within the buffer. This way a source with a centroid near the edge of the brick is not cut off. Instead, the information is still there in the buffer zone. Note: this can still cause issues when nearby sources as separated by the buffer, but this should be rare. 

Bricks are BRICK_WIDTH x BRICK_HEIGHT in size in the non-buffer zone. If the brick is on the mosaic edge, then the buffer will be filled with zeros. Ideally, you want number of pixels along one axis of the mosaic to be integer divisible by the number of bricks along that axis. For example, if your image is 20,000 pixels in one axis, you might chose 2,000 pixels along that axis for the brick size, giving you 10 bricks along that axis. 

If your mosaic is sufficiently small such that bricking is not computationally necessary, then you may set BRICK_WIDTH = MOSAIC_WIDTH and BRICK_HEIGHT = MOSAIC_HEIGHT. A non-zero BRICK_BUFFER is recommended.

Bricks can be made automatically by calling the make_bricks() function:

In [ ]:
interface.make_bricks()

The default arguments of make_bricks are:
- image_type :: Nickname for detection, modeling, or multiband. Default is multiband.
- band :: If a specific band is to be made. Default is None. Optional.
- insert :: True/False. Whether the new bricks should be inserted into existing files.
- skip_psf :: True/False. Can create PSFs at this stage automatically, if already configured.

## Step 3. PSFs

PSFs can either be supplied externally, or be created by The Farmer. Their creation is independent of the brick process, so those steps may be run simultaneously.

The desired PSF type for The Farmer is a .psf file from PSFEx. The Farmer wraps both classical SExtractor (which unlike SEP can output the .LDAC file which PSFEx needs) and PSFEx. 

The aim here is to extract sources from SExtractor and then select pointsource models from the vertical tree in flux radius vs. magnitude space. This is done using the REF_LIMS and VAL_LIMS configurations. The Farmer should automatically rename your PSF files from that which PSFEx outputs to your BAND name, followed by '.psf'. For example, 'sdss_z' would be 'sdss_z.psf', which The Farmer can go find in the DIR_PSFMODELS directory. If your modelling band is the same as one of your other bands, you can simply duplicate that PSF file and rename it to match your modeling nickname in the config file. 

Here we will make use of the make_psf function:

In [ ]:
interface.make_psf()

which has the following arguments:
- image_type :: Same as with make_bricks -- your nickname. Default is multiband.
- band :: Same as with make_bricks -- your band of interest. Default is None, to make all.
- sextractor_only :: Only run SExtractor and plot for output.
- psfex_only :: Plot for output verification, and then only run PSFEx. Requires existing .ldac file.
- override :: True/False. Overwrite existing .psf files?

It is strongly recommended that this be taken in two steps. PLOT must be configured in the config file to be greater than 1 in order to obtain figures from which you can configure the pointsource selection box. 

1. Run SExtractor on the entire mosaics

In [ ]:
interface.make_psf(sextractor_only=True)

2. Using the saved PSF selection image, select the pointsources by configuring the REF_LIMS and VAL_LIMS keywords in the config file.

3. Run PSFEx on the entire mosaics

In [ ]:
interface.make_psf(psfex_only=True)

## Step 4. Detection

Detection happens as part of the make_models routine. Essentially sep (a SExtractor wrapper) is run on the detection brick to determine:
- The number of sources to model
- A guess at the model centroids
- A guess at the flux of the image 

(NB: This last one may be problematic if the detection is not in well-defined units.)

Rejecting a centroid via masking may be done at this point if the use_mask argument is switched on, but is only recommended to mask out bright stars.

See sep for detailed descriptions of the detection parameters, mirrored in the config file.

Source detection is done using:

In [ ]:
interface.detect_sources()

which has the following arguments:
- brick_id :: The brick number you wish to run. If only one brick, then brick_id=1.
- use_mask :: True/False. Set 'True' if the mask should exclude regions from the detection.

The remaining arguments allow the user to inject an existing catalog, plus segmentation map and blobmap to avoid that computation. Not well tested, so not recommended.
- segmap :: Segmap array
- blobmap :: Blobmap arrray.
- catalog :: Catalog table. Can also be set to 'auto' and The Farmer will search for an existing segmentation file and catalog to match the brick id.

## Step 5. Modeling

Modeling can be done on a single band, or several. It is strongly recommended that your model band overlaps with your detection image. If there is no detection, there will be no model!

Sources within a blob are modeled simultaneously to ensure consistency, and each blob may be modelled in parallel if desired, using the NTHREADS>0 keyword in the config file. 

Modeling is done using:

In [ ]:
interface.make_models()

which has the following arguments:
- brick_id :: The brick number you wish to run. If only one brick, then brick_id=1.
- band :: band nickname to be run. Default is MODELING_NICKNAME.
- source_id :: A particular source to be run. Optional. NB: ID's change with the detection and blob configurations!
- blob_id :: A particular blob to be run. Optional. NB: ID's change with the detection and blob configurations!

There are other input arguments, but they are automatically determined. Changing them is not recommended.

For a single model mode:

In [ ]:
interface.make_models()

For multimodel mode, each band is taken in series and their names correspond to the band nicknames as defined in the config file:

In [ ]:
interface.make_models(['sdss_z', 'sdss_y'])

The models will be derived using both images as constraints, with a shared underling shape.

The model catalog per brick will be saved to the DIR_OUTPUT directory. Keep them there!

After the modeling stage has completed, auxillary model and residual maps will then be created, if configured to do so.

----

The Farmer comes with some limited scripts to automatically model and force photometry for a brick of your choosing. Please see bin/tractor_phot.py for details.

## Step 6. Forced Photometry

The last intensive step is the forced photometry. In essence, the best fit model paramters are fixed, and then 'forced' onto the other images of interest leaving only the flux to be optimized.

One can either force the photometry on a single band or many bands. This may be done either in series or in parallel. The series option induces more overhead, and is overall less recommended, but can be used if one wants to allow a model parameter to vary between images (e.g. position). The joint simultaneous forced photometry is the other option, whereby all the multiband images are forced at once.

To do simultaneous forced photometry:

In [ ]:
interface.force_models()

Which has the following arguments:
- brick_id :: The brick number you wish to run. If only one brick, then brick_id=1.
- band :: band nickname to be run. Default is MODELING_NICKNAME.
- source_id :: A particular source to be run. Optional. NB: ID's change with the detection and blob configurations!
- blob_id :: A particular blob to be run. Optional. NB: ID's change with the detection and blob configurations!
- insert :: True/False. Set 'True' if the new photometry should be inserted into the existing model catalogs

-----

Again, see bin/tractor_phot.py for a script to do the modeling and forced photometry in one command line call.

## Step 7. Collecting the catalog

The catalog can be collected by calling the script in /src/tools/collect_catalog.py. Please see the file for details.

The catalog contains the source id, brick id, detection measurements, RA/DEC, pixel positions, as well as shape paramters, model chisq and BIC, raw fluxes, uJy fluxes, AB magnitudes, a valid flag, number of optimization steps taken, and the relevant uncertaintes. 

NB: Catalog files contain the entire config used to produce them in the header!

## Step 8. Auxillary data products

The Farmer is also able to generate mosaics of just models, and hence also chisq and residual mosaics. 

The importance of the residual mosaic is two-fold:

1. It allows you to examine the goodness of your models in aggregate.
2. You can recycle the residual map to detect addtional sources to add to your catalog. You can then force this 'master catalog' back onto all your other bands!
3. You can generate precise masks for your selections by removing bad galaxies. May be required for spatial clustering studies.

They are located in the DIR_INTERIM directory. 

Generation of auxillary images for the entire mosaic is under construction.

NB: It's important to make a cut on the models that are really bad -- negative flux models will show up in the residual as positive flux! See the config.

## Diagnostics

The Farmer supplies several different figures for diagnostic purposes. The PLOT config can be set to 0 (nothing, for n_cpu>1), 1 (basic plots), 2 (intermediate plots), 3 (advanced plots), 4 (all plots). 